# CNA EPSS Data

In [1]:
%%capture
!mkdir -p jsondata
%cd jsondata
!rm *.json 
!rm *.zip 
!wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{2002..2022}.json.zip 
!wget https://epss.cyentia.com/epss_scores-current.csv.gz
!gzip -d epss_scores-current.csv.gz
!unzip -o "*.zip" 

In [2]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import warnings

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))


logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)

In [3]:
row_accumulator = []
for filename in glob.glob('nvdcve-1.1-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data['CVE_Items']:
            cve = entry['cve']['CVE_data_meta']['ID']
            try:
                assigner = entry['cve']['CVE_data_meta']['ASSIGNER']
            except KeyError:
                published_date = 'Missing_Data'
            try:
                published_date = entry['publishedDate']
            except KeyError:
                published_date = 'Missing_Data'
            try:
                attack_vector = entry['impact']['baseMetricV3']['cvssV3']['attackVector']
            except KeyError:
                attack_vector = 'Missing_Data'
            try:
                attack_complexity = entry['impact']['baseMetricV3']['cvssV3']['attackComplexity']
            except KeyError:
                attack_complexity = 'Missing_Data'
            try:
                privileges_required = entry['impact']['baseMetricV3']['cvssV3']['privilegesRequired']
            except KeyError:
                privileges_required = 'Missing_Data'
            try:
                user_interaction = entry['impact']['baseMetricV3']['cvssV3']['userInteraction']
            except KeyError:
                user_interaction = 'Missing_Data'
            try:
                scope = entry['impact']['baseMetricV3']['cvssV3']['scope']
            except KeyError:
                scope = 'Missing_Data'
            try:
                confidentiality_impact = entry['impact']['baseMetricV3']['cvssV3']['confidentialityImpact']
            except KeyError:
                confidentiality_impact = 'Missing_Data'
            try:
                integrity_impact = entry['impact']['baseMetricV3']['cvssV3']['integrityImpact']
            except KeyError:
                integrity_impact = 'Missing_Data'
            try:
                availability_impact = entry['impact']['baseMetricV3']['cvssV3']['availabilityImpact']
            except KeyError:
                availability_impact = 'Missing_Data'
            try:
                base_score = entry['impact']['baseMetricV3']['cvssV3']['baseScore']
            except KeyError:
                base_score = '0.0'
            try:
                base_severity = entry['impact']['baseMetricV3']['cvssV3']['baseSeverity']
            except KeyError:
                base_severity = 'Missing_Data'
            try:
                exploitability_score = entry['impact']['baseMetricV3']['exploitabilityScore']
            except KeyError:
                exploitability_score = 'Missing_Data'
            try:
                impact_score = entry['impact']['baseMetricV3']['impactScore']
            except KeyError:
                impact_score = 'Missing_Data'
            try:
                cwe = entry['cve']['problemtype']['problemtype_data'][0]['description'][0]['value']
            except IndexError:
                cwe = 'Missing_Data'
            try:
                description = entry['cve']['description']['description_data'][0]['value']
            except IndexError:
                description = ''
            new_row = { 
                'CVE': cve, 
                'Published': published_date,
                'AttackVector': attack_vector,
                'AttackComplexity': attack_complexity,
                'PrivilegesRequired': privileges_required,
                'UserInteraction': user_interaction,
                'Scope': scope,
                'ConfidentialityImpact': confidentiality_impact,
                'IntegrityImpact': integrity_impact,
                'AvailabilityImpact': availability_impact,
                'BaseScore': base_score,
                'BaseSeverity': base_severity,
                'ExploitabilityScore': exploitability_score,
                'ImpactScore': impact_score,
                'CWE': cwe,
                'Description': description,
                'Assigner' : assigner
            }
            if not description.startswith('** REJECT **'): # disputed, rejected and other non issues start with '**'
                row_accumulator.append(new_row)
        nvd = pd.DataFrame(row_accumulator)



nvd['Published'] = pd.to_datetime(nvd['Published'])
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)
nvd['BaseScore'] = pd.to_numeric(nvd['BaseScore']);
nvd['BaseScore'] = nvd['BaseScore'].replace(0, np.NaN);
nvdcount = nvd['Published'].count()
nvd['Published'] = pd.to_datetime(nvd['Published']).apply(lambda x: x.date())
nvdcount = nvd['Published'].count()
startdate = date(2000, 1, 1)
enddate  = date.today()
numberofdays = enddate - startdate 
per_day = nvdcount/numberofdays.days


In [4]:

Markdown(f"Total Number of CVEs: **{nvd['CVE'].count()}**<br />Average CVEs Per Day: **{per_day.round(2)}**<br />Average CVSS Score: **{nvd['BaseScore'].mean().round(2)}**")

Total Number of CVEs: **195914**<br />Average CVEs Per Day: **22.93**<br />Average CVSS Score: **7.21**

In [5]:
epss = pd.read_csv('epss_scores-current.csv', skiprows=1)
epss = epss.rename(columns={"cve": "CVE", "epss" : "EPSS"})
epss = epss[['CVE', 'EPSS']]
epss['EPSS'] = epss['EPSS']
epss_nvd = pd.merge(epss, nvd, left_on='CVE', right_on='CVE')
CWE_Value_Counts = epss_nvd['CWE'].value_counts()
CWE_Value_Counts = CWE_Value_Counts.reset_index() 
CWE_Value_Counts = CWE_Value_Counts.rename(columns={"CWE": "CWE Count", "index": "CWE"})
CWE_Value_Counts


,CWE,CWE Count
0,NVD-CWE-Other,27863
1,NVD-CWE-noinfo,22149
2,CWE-79,21337
3,CWE-119,11397
4,CWE-89,9061
...,...,...
348,CWE-334,1
349,CWE-195,1
350,CWE-271,1
351,CWE-760,1


In [11]:
CWE_EPSS_Mean = epss_nvd.groupby('CWE')['EPSS'].mean()
CWE_EPSS_Mean = CWE_EPSS_Mean.reset_index()
CWE_EPSS_Mean = CWE_EPSS_Mean.sort_values(by='EPSS', ascending=False)
CWE_EPSS_Mean = pd.merge(CWE_Value_Counts, CWE_EPSS_Mean, left_on='CWE', right_on='CWE')
CWE_EPSS_Mean.to_csv('epss_mean_with_counts.csv', index=False)
CWE_EPSS_Mean.head(10)

,CWE,CWE Count,EPSS
0,NVD-CWE-Other,27863,0.056797
1,NVD-CWE-noinfo,22149,0.028589
2,CWE-79,21337,0.008461
3,CWE-119,11397,0.138975
4,CWE-89,9061,0.013519
5,CWE-20,8950,0.050946
6,CWE-787,7667,0.025435
7,CWE-200,6694,0.022510
8,CWE-264,5271,0.034923
9,CWE-22,4968,0.051850


In [12]:
CWE_EPSS_Max = epss_nvd.groupby('CWE')['EPSS'].max()
CWE_EPSS_Max = CWE_EPSS_Max.reset_index()
CWE_EPSS_Max = CWE_EPSS_Max.sort_values(by='EPSS', ascending=False)
CWE_EPSS_Max = pd.merge(CWE_Value_Counts, CWE_EPSS_Max, left_on='CWE', right_on='CWE')
CWE_EPSS_Max.to_csv('epss_max_with_counts.csv', index=False)
CWE_EPSS_Max.head(10)

,CWE,CWE Count,EPSS
0,NVD-CWE-Other,27863,0.97559
1,NVD-CWE-noinfo,22149,0.97563
2,CWE-79,21337,0.97508
3,CWE-119,11397,0.97527
4,CWE-89,9061,0.97564
5,CWE-20,8950,0.97578
6,CWE-787,7667,0.97495
7,CWE-200,6694,0.97569
8,CWE-264,5271,0.97536
9,CWE-22,4968,0.97567
